# Week 1, Day 2: RAG Evaluation Pipeline with Ragas

**Sprint**: Week 1 (LLM & RAG Mastery)
**Goal**: Build an evaluation pipeline to measure **Context Recall** and **Faithfulness**.

## 🎯 Problem Statement
How do we know if our RAG system is actually working? 
- **Context Recall**: Did we retrieve the relevant information?
- **Faithfulness**: Is the generated answer true to the retrieved context?

We will use [Ragas](https://github.com/explodinggradients/ragas), a framework that uses an LLM (evaluation-as-a-judge) to score these metrics.

In [ ]:
import sys
import os
from dotenv import load_dotenv

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.append(project_root)

# Load environment variables (OPENAI_API_KEY is required for Ragas by default)
load_dotenv()

import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    context_recall,
    faithfulness,
    answer_relevancy,
)

# Import our RAG components
from src.retrieval.retrieval import HybridRetriever, Document
from src.llm.rag import RAGModel, RetrievalStrategy

## 🧪 1. Setup Data & RAG System

We'll use a small synthetic dataset for this demonstration.

In [ ]:
# 1. Define Knowledge Base
docs = [
    Document(id="1", content="The transformer architecture was introduced in the paper 'Attention Is All You Need' by Vaswani et al. in 2017."),
    Document(id="2", content="BERT (Bidirectional Encoder Representations from Transformers) is designed to pre-train deep bidirectional representations from unlabeled text."),
    Document(id="3", content="GPT-3 is an autoregressive language model that uses deep learning to produce human-like text."),
    Document(id="4", content="RAG (Retrieval-Augmented Generation) combines an information retrieval component with a text generator model."),
]

# 2. Initialize RAG Model (Hybrid)
rag = RAGModel(
    retriever_strategy=RetrievalStrategy.HYBRID,
    generator_model="gpt2", # Using GPT2 for local demo speed, traditionally you'd use a better model
    dense_weight=0.7,
    sparse_weight=0.3
)

print("Indexing documents...")
rag.add_documents(docs)
print("Done.")

## 📝 2. Create Evaluation Dataset

Ragas expects a dataset with columns: `['question', 'answer', 'contexts', 'ground_truth']`.

- **question**: The query.
- **answer**: The generated answer from our RAG pipeline.
- **contexts**: List of strings (retrieved document contents).
- **ground_truth**: List of strings (the correct answer).

In [ ]:
eval_questions = [
    "When was the transformer architecture introduced?",
    "What does BERT stand for?",
    "How does RAG work?"
]

eval_ground_truths = [
    ["The transformer architecture was introduced in 2017."],
    ["BERT stands for Bidirectional Encoder Representations from Transformers."],
    ["RAG combines retrieval with text generation."]
]

# Run Inference to collect 'answer' and 'contexts'
answers = []
contexts = []

print("Running inference...")
for q in eval_questions:
    result = rag.query(q, k=1)
    
    # Store the generated answer
    answers.append(result['response'])
    
    # Store the retrieved contexts (list of strings)
    retrieved_contents = [d['content'] for d in result['retrieved_documents']]
    contexts.append(retrieved_contents)

# Create Dataset
data_dict = {
    "question": eval_questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": eval_ground_truths
}

eval_dataset = Dataset.from_dict(data_dict)
print("Dataset created.")

## 📊 3. Run Evaluation with Ragas

> **Note**: This step requires `OPENAI_API_KEY` to be set in your environment (or `.env` file), as Ragas utilizes an LLM to act as the judge.

In [ ]:
try:
    results = evaluate(
        eval_dataset,
        metrics=[
            context_recall,
            faithfulness,
            answer_relevancy,
            context_precision,
        ],
    )
    
    print("\nEvaluation Results:")
    print(results)
    
    # Convert to Pandas for nicer display
    df_results = results.to_pandas()
    display(df_results)
    
except Exception as e:
    print(f"Evaluation failed (likely due to missing OpenAI Key): {e}")
    print("Ensure OPENAI_API_KEY is set in your .env file.")

## 🧠 Analysis & Interview Talking Points

### **Metric Logic**
1. **Faithfulness**: If low, the model is hallucinating (making things up not in context).
   - *Fix*: Reduce temperature, improve prompt engineering ("Answer only using the provided context").
2. **Context Recall**: If low, the retriever isn't finding the right documents.
   - *Fix*: Tune Hybrid parameters (alpha), improve chunking strategy, or use a better embedding model.

### **Interview Answer**
"In my production RAG system, I set up an automated evaluation pipeline using Ragas directly in my CI/CD. I specifically monitored **Context Recall** to validate my retrieval strategy and **Faithfulness** to ensure the LLM wasn't hallucinating. When Recall dropped, I knew I needed to adjust my hybrid search weights or re-chunk my documents."